# Feature Engineering

## Imports

In [2]:
import sys
# sys.path.append("../src")

# Basic packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random as rd # generating random numbers
import datetime # manipulating date formats
import missingno as mno
# Viz
import matplotlib.pyplot as plt # basic plotting
import seaborn as sns # for prettier plots

from sklearn.preprocessing import StandardScaler

from utils import print_missing_vals
# settings
import warnings
warnings.filterwarnings("ignore")
plt.style.use('ggplot')

In [3]:
peptides = pd.read_csv("../data/train_peptides.csv")
proteins = pd.read_csv("../data/train_proteins.csv")
clinical = pd.read_csv("../data/train_clinical_data.csv")

In [10]:
df_0 = clinical[(clinical.visit_month == 0)][['visit_id','updrs_1']]
print('Train shape:', df_0.shape)
df_0.head()

Train shape: (248, 2)


,visit_id,updrs_1
0,55_0,10.0
13,942_0,3.0
28,1517_0,11.0
38,1923_0,2.0
45,2660_0,2.0


## Peptide Data

### Basic FE

In [7]:
peptide_pepAbundance_ft = peptides.groupby('visit_id').agg(Abe_min=('PeptideAbundance','min'), Abe_max=('PeptideAbundance','max'), Abe_mean=('PeptideAbundance','mean'), Abe_std=('PeptideAbundance','std')).reset_index()
peptide_pepAbundance_ft.head()

,visit_id,Abe_min,Abe_max,Abe_mean,Abe_std
0,10053_0,82.9679,66333900.0,726248.393431,3.535602e+06
1,10053_12,128.4460,73059300.0,737183.385744,3.799654e+06
2,10053_18,108.5000,64711200.0,601466.784320,3.006568e+06
3,10138_12,129.0240,71652400.0,699099.199189,3.379573e+06
4,10138_24,142.6480,123897000.0,732120.888877,4.912602e+06


In [13]:
df_peptides = pd.merge(peptides, df_0, on = 'visit_id', how = 'inner').reset_index()
peptides_PeptideAbundance_updrs = df_peptides.groupby('Peptide').agg(updrs_1_sum = ('updrs_1','mean')).reset_index()
peptides_PeptideAbundance_updrs.head()

,Peptide,updrs_1_sum
0,AADDTWEPFASGK,5.357143
1,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,5.296703
2,AAFTEC(UniMod_4)C(UniMod_4)QAADK,5.305699
3,AANEVSSADVK,5.364780
4,AATGEC(UniMod_4)TATVGKR,5.146497


In [14]:
df_peptides = pd.merge(peptides, peptides_PeptideAbundance_updrs, on = 'Peptide', how = 'left')
peptides_ft = df_peptides.groupby('visit_id').agg(peptides_updrs_1_min=('updrs_1_sum','min'), peptides_updrs_1_max=('updrs_1_sum','max'), peptides_updrs_1_mean=('updrs_1_sum','mean'), peptides_updrs_1_std=('updrs_1_sum','std')).reset_index()
peptides_ft

,visit_id,peptides_updrs_1_min,peptides_updrs_1_max,peptides_updrs_1_mean,peptides_updrs_1_std
0,10053_0,4.878788,5.661972,5.279278,0.092880
1,10053_12,4.816794,5.661972,5.277513,0.097712
2,10053_18,4.297619,5.661972,5.265384,0.116303
3,10138_12,4.297619,5.661972,5.253513,0.126117
4,10138_24,4.297619,5.661972,5.257710,0.123452
...,...,...,...,...,...
1108,8699_24,4.572519,5.661972,5.256902,0.123395
1109,942_12,4.572519,5.661972,5.254323,0.118205
1110,942_24,4.572519,5.652174,5.255565,0.117226
1111,942_48,4.572519,5.652174,5.253489,0.119653


## Protein Data

### Basic FE

In [8]:
proteins_npx_ft = proteins.groupby('visit_id').agg(NPX_min=('NPX','min'), NPX_max=('NPX','max'), NPX_mean=('NPX','mean'), NPX_std=('NPX','std')).reset_index()
proteins_npx_ft.head()

,visit_id,NPX_min,NPX_max,NPX_mean,NPX_std
0,10053_0,2497.84,269126000.0,2.856580e+06,2.131630e+07
1,10053_12,5800.87,270030000.0,2.728871e+06,2.092162e+07
2,10053_18,1334.11,278835000.0,2.509967e+06,1.969453e+07
3,10138_12,2520.24,365582000.0,3.002583e+06,2.516170e+07
4,10138_24,1436.94,396894000.0,3.068891e+06,2.716806e+07


In [11]:
df_proteins = pd.merge(proteins, df_0, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_1','mean')).reset_index()
proteins_Uniprot_updrs.head()

,UniProt,updrs_1_sum
0,O00391,4.971014
1,O00533,5.319588
2,O00584,5.286458
3,O14498,5.217877
4,O14773,5.371585


In [12]:
df_proteins = pd.merge(proteins, proteins_Uniprot_updrs, on = 'UniProt', how = 'left')
proteins_UniProt_ft = df_proteins.groupby('visit_id').agg(proteins_updrs_1_min=('updrs_1_sum','min'), proteins_updrs_1_max=('updrs_1_sum','max'), proteins_updrs_1_mean=('updrs_1_sum','mean'), proteins_updrs_1_std=('updrs_1_sum','std')).reset_index()
proteins_UniProt_ft.head()

,visit_id,proteins_updrs_1_min,proteins_updrs_1_max,proteins_updrs_1_mean,proteins_updrs_1_std
0,10053_0,4.892857,5.601449,5.300548,0.077355
1,10053_12,4.816794,5.652174,5.296073,0.099055
2,10053_18,4.297619,5.652174,5.272617,0.134631
3,10138_12,4.297619,5.652174,5.263118,0.143238
4,10138_24,4.297619,5.652174,5.269522,0.137776


## Overall

### Grouping

In [32]:
# Grouping by patient and month
combined = pd.read_csv('/home/alfred/Code/Kaggle/AMP-Parkinsons/data/engineered/train_protein_peptide_clinical_outlier.csv')

# Group the data by visit_month and patient_id
groups = combined.groupby(['visit_month'])

# Loop through the groups and save each group to a separate CSV file
for group_name, group_data in groups:
    filename = f'/home/alfred/Code/Kaggle/AMP-Parkinsons/data/grouped_by_month_prot/month_{group_name}.csv'
    # print(filename)
    # columns_to_remove_outliers = ['updrs_1','updrs_2','updrs_3','updrs_4']
    # for col in columns_to_remove_outliers:
    #     z_scores = np.abs((group_data[col] - np.mean(group_data[col])) / np.std(group_data[col]))
    #     good_indices = z_scores < 2.5
    #     group_data = group_data[good_indices]
    group_data.to_csv(filename, index=False)

In [66]:
# Grouping by month
combined = pd.read_csv('/home/alfred/Code/Kaggle/AMP-Parkinsons/data/engineered/train_protein_clinical_scaled.csv')

# Group the data by visit_month and patient_id
groups = combined.groupby(['visit_month', 'patient_id'])

# Loop through the groups and save each group to a separate CSV file
for group_name, group_data in groups:
    filename = f'/home/alfred/Code/Kaggle/AMP-Parkinsons/data/grouped_by_id_and_month_prot/{group_name[0]}_{group_name[1]}.csv'
    group_data.to_csv(filename, index=False)

### Pivoting with Proteins

In [4]:
# Load the data from two files
file1 = pd.read_csv('/home/alfred/Code/Kaggle/AMP-Parkinsons/data/imputed/train_clinical_imputed.csv')
file2 = pd.read_csv('/home/alfred/Code/Kaggle/AMP-Parkinsons/data/train_proteins.csv')

# Merge the dataframes based on visit_id, visit_month, and patient_id columns
merged_df = pd.merge(file1, file2, on=['visit_id', 'visit_month', 'patient_id'])

# Pivot the UniProt and NPX columns
pivoted_df = merged_df.pivot(index=['visit_id', 'visit_month', 'patient_id', 'updrs_1', 'updrs_2', 'updrs_3', 'updrs_4', 'upd23b_medication_Off','upd23b_medication_On'], columns='UniProt', values='NPX')

pivoted_df = pivoted_df.reset_index()
pivoted_df = pivoted_df.sort_values(by='patient_id')
# pivoted_df.head()
pivoted_filled_df = pivoted_df.fillna(pivoted_df.mean())
# pivoted_df.to_csv("/home/alfred/Code/Kaggle/AMP-Parkinsons/data/engineered/pivoted.csv")

In [17]:
pivoted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Columns: 236 entries, visit_id to Q9Y6R7
dtypes: float64(231), int64(4), object(1)
memory usage: 1.9+ MB


In [18]:
pivoted_df.describe()

UniProt,visit_month,patient_id,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_medication_Off,upd23b_medication_On,O00391,O00533,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
count,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,736.000000,1.067000e+03,...,915.000000,1031.000000,916.000000,937.000000,1055.000000,1068.000000,870.000000,733.000000,1008.000000,1000.000000
mean,26.744382,32673.328652,6.566479,5.821161,17.344569,1.210674,0.208801,0.214419,11605.680568,5.092013e+05,...,281160.099639,35319.249893,111477.225983,15211.324365,35506.761526,229812.621348,69986.346402,19658.800546,19875.057917,20849.818820
std,22.839426,18612.372744,5.330702,5.931414,14.987692,2.424801,0.406643,0.410611,2815.680292,2.336834e+05,...,135120.685684,18622.313931,24017.506085,4590.724764,12572.302507,102094.316809,26081.467236,6510.064922,5956.448373,10758.041965
min,0.000000,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,873.778000,5.971820e+04,...,9844.570000,4704.100000,47111.700000,4469.080000,1571.400000,13554.300000,3439.810000,3950.620000,2038.500000,3343.200000
25%,6.000000,16574.000000,2.000000,1.000000,2.000000,0.000000,0.000000,0.000000,9722.882500,3.460445e+05,...,197452.000000,21624.850000,94155.150000,12001.100000,26776.150000,157389.500000,54096.900000,15011.500000,15758.200000,13419.625000
50%,24.000000,29365.000000,5.000000,4.000000,16.000000,0.000000,0.000000,0.000000,11482.250000,4.817440e+05,...,271653.000000,33812.500000,109658.500000,14956.700000,35030.000000,213907.500000,67491.050000,18934.600000,19440.900000,18925.850000
75%,48.000000,50611.000000,9.000000,9.000000,28.000000,1.000000,0.000000,0.000000,13371.075000,6.447260e+05,...,357081.000000,47460.000000,126551.750000,18045.600000,42895.150000,284162.250000,84028.275000,23350.300000,23115.400000,26229.050000
max,108.000000,65043.000000,33.000000,29.000000,78.000000,20.000000,1.000000,1.000000,21361.800000,1.806980e+06,...,893220.000000,128338.000000,209158.000000,39060.600000,84868.900000,718141.000000,226139.000000,68511.500000,48743.400000,122699.000000


#### Missing Values of pivoted

In [23]:
def missing_vals(data: pd.DataFrame, amountMissing: int):
    """Prints out the amount of missing data and the percentages

    :param data: Dataset to find missing data in
    :type data: pd.DataFrame
    :param amountMissing: The amount of features to print out
    :type amountMissing: int
    """
    total = data.isnull().sum().sort_values(ascending=False)
    percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data

In [29]:
missing = missing_vals(pivoted_df, 236)
missing.to_csv("/home/alfred/Code/Kaggle/AMP-Parkinsons/data/engineered/missing.csv")

In [30]:
# Filling missing protein values with nans

pivoted_filled_df = pivoted_df.fillna(pivoted_df.mean())

In [26]:
pivoted_filled_df.describe()

UniProt,visit_month,patient_id,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_medication_Off,upd23b_medication_On,O00391,O00533,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
count,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1.068000e+03,...,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000
mean,26.744382,32673.328652,6.566479,5.821161,17.344569,1.210674,0.208801,0.214419,11605.680568,5.092013e+05,...,281160.099639,35319.249893,111477.225983,15211.324365,35506.761526,229812.621348,69986.346402,19658.800546,19875.057917,20849.818820
std,22.839426,18612.372744,5.330702,5.931414,14.987692,2.424801,0.406643,0.410611,2336.925141,2.335738e+05,...,125058.359856,18296.585365,22241.099178,4299.688270,12495.479258,102094.316809,23537.465577,5392.108847,5786.552630,10409.593453
min,0.000000,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,873.778000,5.971820e+04,...,9844.570000,4704.100000,47111.700000,4469.080000,1571.400000,13554.300000,3439.810000,3950.620000,2038.500000,3343.200000
25%,6.000000,16574.000000,2.000000,1.000000,2.000000,0.000000,0.000000,0.000000,10566.650000,3.465792e+05,...,211038.250000,22014.375000,97283.050000,12464.175000,26967.625000,157389.500000,57822.200000,16819.975000,15925.825000,13847.925000
50%,24.000000,29365.000000,5.000000,4.000000,16.000000,0.000000,0.000000,0.000000,11605.680568,4.820360e+05,...,281160.099639,34600.650000,111477.225983,15211.324365,35184.850000,213907.500000,69986.346402,19658.800546,19838.550000,19846.150000
75%,48.000000,50611.000000,9.000000,9.000000,28.000000,1.000000,0.000000,0.000000,12435.600000,6.444495e+05,...,334399.500000,46993.250000,122562.750000,17450.300000,42853.450000,284162.250000,79080.325000,21212.325000,22860.850000,25674.975000
max,108.000000,65043.000000,33.000000,29.000000,78.000000,20.000000,1.000000,1.000000,21361.800000,1.806980e+06,...,893220.000000,128338.000000,209158.000000,39060.600000,84868.900000,718141.000000,226139.000000,68511.500000,48743.400000,122699.000000


### Pivoting with Peptides

In [6]:
# Load the data from two files
file1 = pd.read_csv('/home/alfred/Code/Kaggle/AMP-Parkinsons/data/engineered/train_protein_clinical.csv')
file2 = pd.read_csv('/home/alfred/Code/Kaggle/AMP-Parkinsons/data/train_peptides.csv')

# Merge the dataframes based on visit_id, visit_month, and patient_id columns
merged_df = pd.merge(file1, file2, on=['visit_id', 'visit_month', 'patient_id'])

title = 'visit_id,visit_month,patient_id,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_medication_Off,upd23b_medication_On,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,O60888,O75144,O75326,O94919,P00441,P00450,P00734,P00736,P00738,P00746,P00747,P00748,P00751,P01008,P01009,P01011,P01019,P01023,P01024,P01031,P01033,P01034,P01042,P01344,P01591,P01594,P01608,P01621,P01717,P01780,P01833,P01834,P01857,P01859,P01860,P01861,P01876,P01877,P02452,P02647,P02649,P02652,P02655,P02656,P02671,P02675,P02679,P02747,P02748,P02749,P02750,P02751,P02753,P02760,P02763,P02765,P02766,P02768,P02774,P02787,P02790,P02792,P04004,P04075,P04156,P04180,P04196,P04207,P04211,P04216,P04217,P04275,P04406,P04433,P05060,P05067,P05090,P05155,P05156,P05408,P05452,P05546,P06310,P06396,P06454,P06681,P06727,P07195,P07225,P07333,P07339,P07602,P07711,P07858,P07998,P08123,P08133,P08253,P08294,P08493,P08571,P08603,P08637,P08697,P09104,P09486,P09871,P10451,P10643,P10645,P10909,P11142,P11277,P12109,P13473,P13521,P13591,P13611,P13671,P13987,P14174,P14314,P14618,P16035,P16070,P16152,P16870,P17174,P17936,P18065,P19021,P19652,P19823,P19827,P20774,P20933,P23083,P23142,P24592,P25311,P27169,P30086,P31997,P32754,P35542,P36222,P36955,P36980,P39060,P40925,P41222,P43121,P43251,P43652,P49588,P49908,P51884,P54289,P55290,P60174,P61278,P61626,P61769,P61916,P80748,P98160,Q02818,Q06481,Q08380,Q12805,Q12841,Q12907,Q13283,Q13332,Q13449,Q13451,Q13740,Q14118,Q14508,Q14515,Q14624,Q15904,Q16270,Q16610,Q562R1,Q6UX71,Q6UXB8,Q6UXD5,Q7Z3B1,Q7Z5P9,Q8IWV7,Q8N2S1,Q8NBJ4,Q8NE71,Q92520,Q92823,Q92876,Q96BZ4,Q96KN2,Q96PD5,Q96S96,Q99435,Q99674,Q99683,Q99829,Q99832,Q99969,Q9BY67,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7'

cols = title.split(',')
# Pivot the UniProt and NPX columns
pivoted_peptide_df = merged_df.pivot(index=cols, columns='Peptide', values='PeptideAbundance')

pivoted_peptide_df = pivoted_peptide_df.reset_index()
pivoted_peptide_df = pivoted_peptide_df.sort_values(by=['patient_id', 'visit_month'])
pivoted_peptide_df = pivoted_peptide_df.fillna(pivoted_peptide_df.mean())
pivoted_peptide_df.head()

Peptide,visit_id,visit_month,patient_id,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_medication_Off,upd23b_medication_On,O00391,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
814,55_0,0,55,10.0,6.0,15.0,1.0,0,0,11254.3,...,201158.0,16492.3,3.810270e+06,106894.0,580667.0,131155.0,165851.0,437305.0,46289.2,14898.4
817,55_6,6,55,8.0,10.0,34.0,1.0,0,0,13163.6,...,171079.0,13198.8,4.119520e+06,113385.0,514861.0,103512.0,144607.0,457891.0,40047.7,20703.9
815,55_12,12,55,10.0,10.0,41.0,0.0,0,1,15257.6,...,231772.0,17873.8,5.474140e+06,116286.0,711815.0,136943.0,181763.0,452253.0,54725.1,21841.1
816,55_36,36,55,17.0,18.0,51.0,0.0,0,1,13530.8,...,185290.0,18580.5,2.659660e+06,90936.9,679163.0,128593.0,203680.0,498621.0,52792.7,13973.7
1067,942_6,6,942,8.0,2.0,21.0,0.0,0,0,11218.7,...,226314.0,6399.8,3.938239e+06,57571.4,480951.0,80001.2,79661.9,573300.0,48005.8,15674.1


In [41]:
pivoted_peptide_df.describe()

Peptide,visit_month,patient_id,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_medication_Off,upd23b_medication_On,O00391,O00533,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
count,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1.068000e+03,...,1068.000000,1068.000000,1.068000e+03,1068.000000,1.068000e+03,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000
mean,26.744382,32673.328652,6.566479,5.821161,17.344569,1.210674,0.208801,0.214419,11605.680568,5.092013e+05,...,215475.070900,8996.806715,3.938239e+06,68394.594621,6.130732e+05,92668.876265,125754.068949,472235.953491,46987.825294,21042.291924
std,22.839426,18612.372744,5.330702,5.931414,14.987692,2.424801,0.406643,0.410611,2336.925141,2.335738e+05,...,50313.883464,2961.030625,1.620814e+06,57816.007323,4.945741e+05,30091.801489,38243.867433,130017.630872,13069.455974,9105.018018
min,0.000000,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,873.778000,5.971820e+04,...,12164.300000,258.249000,1.624640e+05,884.260000,7.494660e+03,869.898000,991.452000,11371.200000,6362.490000,868.903000
25%,6.000000,16574.000000,2.000000,1.000000,2.000000,0.000000,0.000000,0.000000,10566.650000,3.465792e+05,...,187394.000000,7087.587500,2.941620e+06,34201.850000,2.949230e+05,70517.575000,100952.000000,385090.250000,38471.475000,16230.625000
50%,24.000000,29365.000000,5.000000,4.000000,16.000000,0.000000,0.000000,0.000000,11605.680568,4.820360e+05,...,216340.000000,8996.806715,3.872205e+06,59685.700000,4.845965e+05,88893.050000,124152.000000,464453.000000,46863.300000,21042.291924
75%,48.000000,50611.000000,9.000000,9.000000,28.000000,1.000000,0.000000,0.000000,12435.600000,6.444495e+05,...,246187.500000,10330.775000,4.631290e+06,80915.175000,7.591085e+05,110132.500000,149156.500000,548241.250000,53433.650000,24460.275000
max,108.000000,65043.000000,33.000000,29.000000,78.000000,20.000000,1.000000,1.000000,21361.800000,1.806980e+06,...,409939.000000,27670.500000,1.385550e+07,712856.000000,3.984710e+06,251526.000000,264224.000000,948416.000000,107220.000000,70020.800000


In [42]:
pivoted_peptide_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1068 entries, 817 to 1025
Columns: 1204 entries, visit_id to YYWGGQYTWDMAK
dtypes: float64(1199), int64(4), object(1)
memory usage: 9.8+ MB


### Standardize Values

In [7]:
# select columns to standardize
cols_to_standardize = list(pivoted_peptide_df.columns)

cols_to_remove = ['visit_id','visit_month','patient_id','updrs_1','updrs_2','updrs_3','updrs_4','upd23b_medication_Off','upd23b_medication_On']

cols_to_standardize = [x for x in cols_to_standardize if x not in cols_to_remove]

# create StandardScaler object
scaler = StandardScaler()

# fit scaler on selected columns
scaler.fit(pivoted_peptide_df[cols_to_standardize])

# transform selected columns
pivoted_peptide_df[cols_to_standardize] = scaler.transform(pivoted_peptide_df[cols_to_standardize])
pivoted_peptide_df.head()

Peptide,visit_id,visit_month,patient_id,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_medication_Off,upd23b_medication_On,O00391,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
814,55_0,0,55,10.0,6.0,15.0,1.0,0,0,-0.150431,...,-0.284688,2.532566,-7.899077e-02,0.666207,-0.065554,1.279556,1.048945,-0.268789,-0.053480,-0.675097
817,55_6,6,55,8.0,10.0,34.0,1.0,0,0,0.666966,...,-0.882796,1.419763,1.118978e-01,0.778530,-0.198672,0.360504,0.493197,-0.110383,-0.531267,-0.037183
815,55_12,12,55,10.0,10.0,41.0,0.0,0,1,1.563435,...,0.324057,2.999345,9.480547e-01,0.828730,0.199744,1.471991,1.465207,-0.153766,0.592289,0.087774
816,55_36,36,55,17.0,18.0,51.0,0.0,0,1,0.824169,...,-0.600216,3.238124,-7.892198e-01,0.390080,0.133692,1.194377,2.038561,0.203029,0.444364,-0.776704
1067,942_6,6,942,8.0,2.0,21.0,0.0,0,0,-0.165671,...,0.215527,-0.877473,-2.874355e-16,-0.187288,-0.267268,-0.421165,-1.205782,0.777674,0.077926,-0.589862


In [67]:
# select columns to standardize
cols_to_standardize = list(pivoted_filled_df.columns)

cols_to_remove = ['visit_id','visit_month','patient_id','updrs_1','updrs_2','updrs_3','updrs_4','upd23b_medication_Off','upd23b_medication_On']

cols_to_standardize = [x for x in cols_to_standardize if x not in cols_to_remove]

# create StandardScaler object
scaler = StandardScaler()

# fit scaler on selected columns
scaler.fit(pivoted_filled_df[cols_to_standardize])

# transform selected columns
pivoted_filled_df[cols_to_standardize] = scaler.transform(pivoted_filled_df[cols_to_standardize])
pivoted_filled_df.head()

UniProt,visit_id,visit_month,patient_id,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_medication_Off,upd23b_medication_On,O00391,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
817,55_6,6,55,8.0,10.0,34.0,1.0,0,0,0.666966,...,0.996129,-0.272673,-0.103606,1.928595,1.292996,1.372486,0.000000,0.607913,-0.372167,-0.404347
816,55_36,36,55,17.0,18.0,51.0,0.0,0,1,0.824169,...,0.179495,0.703693,-0.075716,2.028836,2.779646,1.447746,0.212091,2.240210,0.399638,0.083355
815,55_12,12,55,10.0,10.0,41.0,0.0,0,1,1.563435,...,0.182343,0.421421,0.154911,1.547063,2.089048,0.869607,-0.179532,1.769079,1.497455,-0.149828
814,55_0,0,55,10.0,6.0,15.0,1.0,0,0,-0.150431,...,0.674520,0.011415,-0.650975,1.840804,2.034155,1.752979,0.000000,1.873985,0.684431,-0.182256
1067,942_6,6,942,8.0,2.0,21.0,0.0,0,0,-0.165671,...,-0.222297,-0.431291,-0.795321,0.521113,-1.137557,0.752680,0.524905,0.808949,-0.161754,-0.538085


### Remove Outliers

In [31]:
columns_to_remove_outliers = ['updrs_1','updrs_2','updrs_3','updrs_4']


for col in columns_to_remove_outliers:
    z_scores = np.abs((pivoted_peptide_df[col] - np.mean(pivoted_peptide_df[col])) / np.std(pivoted_peptide_df[col]))
    good_indices = z_scores < 3
    pivoted_peptide_df = pivoted_peptide_df[good_indices]

### Ending

#### All feature engineering

In [30]:
# Load the data from two files
file1 = pd.read_csv('/home/alfred/Code/Kaggle/AMP-Parkinsons/data/imputed/train_clinical_imputed.csv')
file2 = pd.read_csv('/home/alfred/Code/Kaggle/AMP-Parkinsons/data/train_proteins.csv')

# Merge the dataframes based on visit_id, visit_month, and patient_id columns
merged_df = pd.merge(file1, file2, on=['visit_id', 'visit_month', 'patient_id'])

# Pivot the UniProt and NPX columns
pivoted_df = merged_df.pivot(index=['visit_id', 'visit_month', 'patient_id', 'updrs_1', 'updrs_2', 'updrs_3', 'updrs_4', 'upd23b_medication_Off','upd23b_medication_On'], columns='UniProt', values='NPX')

pivoted_df = pivoted_df.reset_index()
pivoted_df = pivoted_df.sort_values(by='patient_id')
# pivoted_df.head()
pivoted_filled_df = pivoted_df.fillna(pivoted_df.mean())


# Load the data from two files
file1 = pivoted_filled_df
file2 = pd.read_csv('/home/alfred/Code/Kaggle/AMP-Parkinsons/data/train_peptides.csv')

# Merge the dataframes based on visit_id, visit_month, and patient_id columns
merged_df = pd.merge(file1, file2, on=['visit_id', 'visit_month', 'patient_id'])

title = 'visit_id,visit_month,patient_id,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_medication_Off,upd23b_medication_On,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,O60888,O75144,O75326,O94919,P00441,P00450,P00734,P00736,P00738,P00746,P00747,P00748,P00751,P01008,P01009,P01011,P01019,P01023,P01024,P01031,P01033,P01034,P01042,P01344,P01591,P01594,P01608,P01621,P01717,P01780,P01833,P01834,P01857,P01859,P01860,P01861,P01876,P01877,P02452,P02647,P02649,P02652,P02655,P02656,P02671,P02675,P02679,P02747,P02748,P02749,P02750,P02751,P02753,P02760,P02763,P02765,P02766,P02768,P02774,P02787,P02790,P02792,P04004,P04075,P04156,P04180,P04196,P04207,P04211,P04216,P04217,P04275,P04406,P04433,P05060,P05067,P05090,P05155,P05156,P05408,P05452,P05546,P06310,P06396,P06454,P06681,P06727,P07195,P07225,P07333,P07339,P07602,P07711,P07858,P07998,P08123,P08133,P08253,P08294,P08493,P08571,P08603,P08637,P08697,P09104,P09486,P09871,P10451,P10643,P10645,P10909,P11142,P11277,P12109,P13473,P13521,P13591,P13611,P13671,P13987,P14174,P14314,P14618,P16035,P16070,P16152,P16870,P17174,P17936,P18065,P19021,P19652,P19823,P19827,P20774,P20933,P23083,P23142,P24592,P25311,P27169,P30086,P31997,P32754,P35542,P36222,P36955,P36980,P39060,P40925,P41222,P43121,P43251,P43652,P49588,P49908,P51884,P54289,P55290,P60174,P61278,P61626,P61769,P61916,P80748,P98160,Q02818,Q06481,Q08380,Q12805,Q12841,Q12907,Q13283,Q13332,Q13449,Q13451,Q13740,Q14118,Q14508,Q14515,Q14624,Q15904,Q16270,Q16610,Q562R1,Q6UX71,Q6UXB8,Q6UXD5,Q7Z3B1,Q7Z5P9,Q8IWV7,Q8N2S1,Q8NBJ4,Q8NE71,Q92520,Q92823,Q92876,Q96BZ4,Q96KN2,Q96PD5,Q96S96,Q99435,Q99674,Q99683,Q99829,Q99832,Q99969,Q9BY67,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7'

cols = title.split(',')
# Pivot the UniProt and NPX columns
pivoted_peptide_df = merged_df.pivot(index=cols, columns='Peptide', values='PeptideAbundance')

pivoted_peptide_df = pivoted_peptide_df.reset_index()
pivoted_peptide_df = pivoted_peptide_df.sort_values(by=['patient_id', 'visit_month'])
pivoted_peptide_df = pivoted_peptide_df.fillna(pivoted_peptide_df.mean())


# select columns to standardize
cols_to_standardize = list(pivoted_peptide_df.columns)

cols_to_remove = ['visit_id','visit_month','patient_id','updrs_1','updrs_2','updrs_3','updrs_4','upd23b_medication_Off','upd23b_medication_On']

cols_to_standardize = [x for x in cols_to_standardize if x not in cols_to_remove]

# create StandardScaler object
scaler = StandardScaler()

# fit scaler on selected columns
scaler.fit(pivoted_peptide_df[cols_to_standardize])

# transform selected columns
pivoted_peptide_df[cols_to_standardize] = scaler.transform(pivoted_peptide_df[cols_to_standardize])

In [24]:
pivoted_peptide_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 967 entries, 814 to 1026
Columns: 1204 entries, visit_id to YYWGGQYTWDMAK
dtypes: float64(1199), int64(4), object(1)
memory usage: 8.9+ MB


#### To CSV

In [5]:
pivoted_filled_df.to_csv("/home/alfred/Code/Kaggle/AMP-Parkinsons/data/engineered/train_protein_clinical.csv", index=False)

In [68]:
pivoted_filled_df.to_csv("/home/alfred/Code/Kaggle/AMP-Parkinsons/data/engineered/train_protein_clinical_scaled.csv", index=False)

In [10]:
pivoted_peptide_df.to_csv("/home/alfred/Code/Kaggle/AMP-Parkinsons/data/engineered/train_protein_peptide_clinical.csv", index=False)

In [9]:
pivoted_peptide_df.to_csv("/home/alfred/Code/Kaggle/AMP-Parkinsons/data/engineered/train_protein_peptide_clinical_scaled.csv", index=False)

In [25]:
pivoted_peptide_df.to_csv("/home/alfred/Code/Kaggle/AMP-Parkinsons/data/engineered/train_protein_peptide_clinical_outlier.csv", index=False)